# Homodyne detection by a coherent state as local oscillator and a squeezed state

Use the multihead (2-head) gates in the phase space 
to create a network that represents a single mode squeezed vacuum
and a coherent state interferring on a Beam splitter

The second derivatives of the chi matrix are used to evaluate the expected number of photons in each channels


<img src="../img/BS.png" width="400" height="400" /> 

<img src="../img/BellBS.png" width="800" height="200" />

<img src="../img/logo_circular.png" width="20" height="20" />@by claudio<br>


nonlinearxwaves@gmail.com<br>
@created 7 january 2021<br>
@version 1 may 2023

In [1]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
from thqml import phasespace as ps
from thqml.utilities import utilities
#import matplotlib.pyplot as plt
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow import keras
#import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as kb

2023-05-01 11:04:35.066344: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-01 11:04:35.066365: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf.keras.backend.clear_session()

In [3]:
np.set_printoptions(precision=2)

## Dimension (number of modes times 2, N=2n)

In [4]:
N = 4

## Build vacuum by the Gaussian state

In [5]:
vacuum = ps.VacuumLayer(N)

2023-05-01 11:04:37.529592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-01 11:04:37.529622: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-01 11:04:37.529644: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (x1): /proc/driver/nvidia/version does not exist
2023-05-01 11:04:37.529976: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Squeezed mode on mode 0

In [6]:
ra=0.1;
phia=0;

In [7]:
squeezer=ps.SingleModeSqueezerLayer(N, r_np=ra, theta_np=phia, n_squeezed=0, trainable=False)

## Coherent state (Displacer on mode 1)

In [8]:
A_np = 10
lambda_np = 0
lambda_np = np.pi*0.5
#theta =0
dinput=np.zeros((N,1));
dinput[2]=np.sqrt(2)*A_np*np.cos(lambda_np);
dinput[3]=np.sqrt(2)*A_np*np.sin(lambda_np);
displacer=ps.DisplacementLayerConstant(dinput)

## Total number of photons

In [9]:
n_total= A_np**2+np.sinh(ra)**2; print(n_total)

100.01003337780953


## Beam splitter

In [10]:
bs0 = ps.BeamSplitterLayer(N, theta=np.pi/4, phi0=0, phi1=0, n_0=0, n_1=1,  trainable_theta=True)

In [11]:
M, MI = bs0.get_M(); tf.print(M)

[[0.707106769 0 -0.707106769 0]
 [0 0.707106769 0 -0.707106769]
 [0.707106769 0 0.707106769 0]
 [0 0.707106769 0 0.707106769]]


### Dummy training points (not used for training in this example)

In [12]:
Nbatch=10
gtarget=np.eye(N)
dtarget=2.4*np.ones((N,1))
xtrain = np.random.rand(Nbatch, N)-0.5
ytrain =np.zeros_like(xtrain)
dtrain = np.zeros((Nbatch,N))
gtrain = np.zeros((Nbatch,N,N))
for j in range(Nbatch):
    for i in range(N):
        dtrain[j,i]=dtarget[i]
        for k in range(N):
            gtrain[j,i,k]=gtarget[i,k]

# Build the model without the BS

In [13]:
xin = tf.keras.layers.Input(N)
x1, a1 = squeezer(xin)
x0, a0 = displacer(x1,a1)
chir, chii = vacuum(x0, a0)
modelPC = tf.keras.Model(inputs = xin, outputs=[chir, chii])

### Add the layer to compute the average photon number

In [14]:
photon_counter=ps.PhotonCountingLayer(N) # define the layer
n_out = photon_counter(chir,chii, modelPC);  # define the output tensor
NphotonPC = tf.keras.Model(inputs = xin, outputs=n_out) # define the model with inputs and ouputs

Compute the number of photons in each mode (the number of photons does not depend on xtrain)

In [15]:
print(NphotonPC(xtrain)); 

tf.Tensor([[1.e-02 1.e+02]], shape=(1, 2), dtype=float32)


Note that we have 0 photons in modes 0 and A_np^2 in mode 1

# Build the model with the BS

In [16]:
xin = tf.keras.layers.Input(N)
x2, a2 = bs0(xin)
x1, a1 = squeezer(x2,a2)
x0, a0 = displacer(x1,a1)
chir, chii = vacuum(x0, a0)
BellBS = tf.keras.Model(inputs = xin, outputs=[chir, chii])

### add the photon counter

In [17]:
photon_counter=ps.PhotonCountingLayer(N)
n_out = photon_counter(chir,chii, BellBS)
Nphoton = tf.keras.Model(inputs = xin, outputs=[n_out])
print(Nphoton(xtrain)) 

tf.Tensor([[50.01 50.01]], shape=(1, 2), dtype=float32)


Remark : the input photons are divided in the two branchs of the interferometer

# Define a custom layer to subtract the photon counts on the two channels

In [18]:
@tf.function
def differential_detector(nphoton, alpha = 1):
    n1 = tf.slice(nphoton, [0,0], [1,1])
    n2 = tf.slice(nphoton, [0,1], [1,1])
    return (n2-n1)/(np.sqrt(2)*alpha)

In [19]:
print(differential_detector(Nphoton(xtrain), A_np))

tf.Tensor([[0.]], shape=(1, 1), dtype=float32)


# Evaluate the uncertainties by DifferentialGaussianLayer 

In [20]:
Diff=ps.DifferentialGaussianLayer(N)
nboson, Dn, Dn2 = Diff(chir,chii, BellBS)
HModel = tf.keras.Model(inputs = xin, outputs=[nboson, Dn, Dn2])
print(HModel(xtrain)) 

[<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[50.01, 50.01]], dtype=float32)>, <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0., 0.],
       [0., 0.]], dtype=float32)>, <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[  0.  , 122.15],
       [122.15,   0.  ]], dtype=float32)>]


## Compare with the theoretical value

In [21]:
x2theory= (A_np**2)*(np.exp(-2*ra)*(np.cos(lambda_np-0.5*phia))**2+np.exp(2*ra)*(np.sin(lambda_np-0.5*phia))**2)

In [22]:
print(x2theory)

122.14027581601698


This is equal within numerical precision to diagonal element in Dn2

Dx^2

In [23]:
x2theory/(2*A_np**2)

0.6107013790800849

Smaller or larger than 0.5 if the mode is squeezed (depends on ra, the two quadrature correspond to phia=0 and phia=np.pi/2).
The proper value of the quadrature is retrieved setting lambda=phia/2, so that lambda =0 for phia=0, and lambda =pi/4 for phia=pi/2
For a coherent state (ra=0.0) it is equal to 0.5